# Import

In [3]:
import os
import pandas as pd
import pretty_midi
import librosa
import librosa.display
import matplotlib.pyplot as plt
import torch
import numpy as np
import subprocess
from basic_pitch_torch.inference import predict
from tqdm.notebook import tqdm

### Path to dataset

In [1]:
DATA_DIR = "C:/Users/milan/Documents/oc/p7/maestro-v3.0.0/maestro-v3.0.0/"

### Loding CSV with all files path as feature and descrition of the music

In [4]:
csv_path = os.path.join(DATA_DIR, "maestro-v3.0.0.csv")
df = pd.read_csv(csv_path)
display(df.head())

,canonical_composer,canonical_title,split,year,midi_filename,audio_filename,duration
0,Alban Berg,Sonata Op. 1,train,2008,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MI...,759.518471
1,Alban Berg,Sonata Op. 1,train,2018,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R...,698.661160
2,Alban Berg,Sonata Op. 1,train,2017,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-...,464.649433
3,Alexander Scriabin,"24 Preludes Op. 11, No. 13-24",train,2004,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MI...,872.640588
4,Alexander Scriabin,"3 Etudes, Op. 65",validation,2006,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MI...,397.857508


In [ ]:
def generate_midi_basic_pitch(audio_path):
    '''
    make prediction on audio file and save midi file
    '''
    save_path = audio_path.replace(".wav", "_bp.mid")
    audio_path = DATA_DIR + audio_path
    midi_output_path = audio_path.replace(".wav", "_bp.mid")
    if os.path.exists(midi_output_path):
        print(f"Déjà existant : {midi_output_path}")
        return save_path
    try:
        model_output, midi_data, note_events = predict(audio_path)
        midi_data.write(midi_output_path)
        return save_path
    except Exception as e:
        print(f"Erreur avec {audio_path}: {e}")
        return None

In [ ]:
def generate_midi_melodia(audio_path):
    '''
    make prediction on audio file and save midi file
    '''
    save_path = audio_path.replace(".wav", ".mid")
    audio_path = DATA_DIR + audio_path
    midi_output_path = audio_path.replace(".wav", ".mid") 
    if os.path.exists(midi_output_path):
        print(f"Déjà existant : {midi_output_path}")
        return save_path
    try:
        cmd = f'sonic-annotator -d vamp:mtg-melodia:melodia -w midi "{audio_path}"'
        subprocess.run(cmd, shell=True, check=True)
        return save_path
    except Exception as e:
        print(f"Erreur avec {audio_path}: {e}")
        return None

### Make prediction for Basic pitch, save the file and put the path in the dataframe

In [ ]:
df["midi_bp"] = df["audio_filename"].apply(generate_midi_basic_pitch)


### Look if all files are not empty

In [ ]:
print(df["midi_bp"].notnull().sum())
print(df.describe())

1204
              year     duration
count  1276.000000  1276.000000
mean   2011.302508   560.463595
std       4.353338   443.057431
min    2004.000000    45.155208
25%    2008.000000   261.981576
50%    2011.000000   429.168241
75%    2015.000000   685.018817
max    2018.000000  2624.663508


### Make prediction for Melodia, save the file and put the path in the dataframe

In [ ]:
df["midi_melodia"] = df["audio_filename"].apply(generate_midi_melodia)

### Look if all files are not empty

In [ ]:
print(df["midi_melodia"].notnull().sum())
print(df.describe())

1275
              year     duration
count  1276.000000  1276.000000
mean   2011.302508   560.463595
std       4.353338   443.057431
min    2004.000000    45.155208
25%    2008.000000   261.981576
50%    2011.000000   429.168241
75%    2015.000000   685.018817
max    2018.000000  2624.663508


### Delete files where prediction was empty

In [ ]:
df.dropna().to_csv("maestro_and_predict.csv", index=False)

In [ ]:
midi_df = df.dropna()

In [ ]:
print(midi_df["midi_bp"][0])

2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2_bp.mid


In [ ]:
def extract_notes(midi_path):
        """ Extract notes from a midi file. """
        midi_data = pretty_midi.PrettyMIDI(DATA_DIR + midi_path)
        notes = []
        for instrument in midi_data.instruments:
            for note in instrument.notes:
                notes.append((note.pitch, note.start, note.end, note.velocity))
        return pd.DataFrame(notes, columns=["Note", "Start_Time", "End_Time", "Velocity"])

def calculate_metrics(original, pred):
        """ Calculate F1 Score, recall and precision. """
        total_original = len(original)
        total_pred = len(pred)
        matched_notes = original.merge(pred, on="Note", how="inner")
        true_positives = len(matched_notes)

        precision = true_positives / total_pred if total_pred > 0 else 0
        recall = true_positives / total_original if total_original > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return f1_score, precision, recall

## Create a new dataframe where features are path as primary key, the metrics, one dataframe per predict and the original midi 

In [ ]:
results = []
for _, row in tqdm(midi_df.iterrows(), total=len(midi_df), desc="Traitement des fichiers MIDI"):
    original_notes = extract_notes(row["midi_filename"])
    if original_notes.empty:
        continue
    basic_pitch_notes = extract_notes(row["midi_bp"])
    melodia_notes = extract_notes(row["midi_melodia"])

    f1_bp, precision_bp, recall_bp = calculate_metrics(original_notes, basic_pitch_notes)
    f1_mel, precision_mel, recall_mel = calculate_metrics(original_notes, melodia_notes)

    results.append({
        "morceau": row["audio_filename"],
        "original_notes": original_notes,
        "basic_pitch_notes": basic_pitch_notes,
        "melodia_notes": melodia_notes,
        "F1 Score BP": f1_bp,
        "Précision BP": precision_bp,
        "Rappel BP": recall_bp,
        "F1 Score Mel": f1_mel,
        "Précision Mel": precision_mel,
        "Rappel Mel": recall_mel,
    })

final_df = pd.DataFrame(results)

### Save the results

In [ ]:
final_df.to_csv("metrics.csv", index=False)